## Drone Detection

In [ ]:
# essential imports
import os
import csv
import cv2
import json
import numpy as np

# plotting and display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, clear_output

# YOLO model from ultralytics package
from ultralytics import YOLO

# custom utilities
from utils import *

# reflect changes in src code immediately w/o restarting kernel
%load_ext autoreload
%autoreload 2

# path to all datasets
data_path = './data/drone-tracking-datasets/'

#### Set device and load model

In [ ]:
# set the device depending on available GPU
device = set_device()

In [ ]:
# choose model size: n for nano, s for small, m for medium, l for large, x for extra large
model_size = 'm'  

# load pretrained YOLOv8 model
model = YOLO(f"yolov8{model_size}.pt")

#### Select dataset and video

In [ ]:
# select dataset and camera
dataset_num = 3
cam_num = 4
video_path = os.path.join(data_path, f'dataset{dataset_num}/cam{cam_num}.mp4')

# get sample frame and store width and height
cap = cv2.VideoCapture(video_path)
ret, sample_frame = cap.read()
cap.release()
sample_frame_rgb = cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB)
frame_height, frame_width, _ = sample_frame.shape
print(f"Frame size: {frame_width} x {frame_height}")
plt.imshow(sample_frame_rgb)

#### Run detection with YOLO

In [ ]:
# select tracker
tracker = "sort.yaml"  # or 'bytetrack.yaml', 'strongsort.yaml'

# pick either show or stream

# run detection on video
results = model.predict(
    source=video_path, 
    device=device,
    tracker=tracker,

    #show=True, # display live output in external window (do not return generator)
    save=True,  # save output video with bounding boxes to runs/detect/
    stream=True  # return a generator that yields results for each frame, doesn't store whole video in memory
)

In [ ]:
# haven't finetuned YOLO yet for drone detection class 
# in the meantime, model identifies drones as either airplanes, birds, kites
print(model.names[4])
print(model.names[14])
print(model.names[33])

In [ ]:
# only keep detections of target classes (airplane, bird, kite in that order)
target_classes = torch.tensor([4, 14, 33], device=device, dtype=torch.float32)

# save trajectory to CSV
csv_file = f"trajectory_dataset{dataset_num}_cam{cam_num}.csv"

trajectory = []  # list to store (frame_id, x_center, y_center)

for frame_id, result in enumerate(results, start=1):

    # filter boxes to only target classes
    mask = torch.isin(result.boxes.cls, target_classes)
    target_boxes = result.boxes[mask]
    
    if len(target_boxes) > 0:
        # Choose the most confident detection among these classes
        best_idx = target_boxes.conf.argmax()
        box = target_boxes[best_idx].xywh[0]
        x_center, y_center = box[0].item(), box[1].item()
    else:
        x_center, y_center = float('nan'), float('nan')

    trajectory.append([frame_id, x_center, y_center])

# save to CSV
with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame_id", "x_center", "y_center"])
    writer.writerows(trajectory)

print(f"Trajectory saved to {csv_file}")

#### Plot 2D trajectory

In [ ]:
plot_2d_trajectory(trajectory, sample_frame_rgb)

#### Plot GT 3D trajectory

In [ ]:
# load text file
data_path = './data/drone-tracking-datasets/'
dataset_num = 3
trajectory_path = os.path.join(data_path, f'dataset{dataset_num}/trajectory/rtk.txt')

In [ ]:
plot_3d_trajectory_static(trajectory_path)

In [ ]:
plot_3d_trajectory_interactive(trajectory_path)

#### Dataset 3 - Camera Network Details

In [ ]:


# positions of cameras in world coordinates
''' 
44.535 11.56253333 -1.1467
4.3962 -54.27346667 4.0965
-42.5242 -21.00086667 -1.7639
40.1912 44.79053333 -0.9379
-34.8458 -44.00906667 1.4179
-11.7524 62.93033333 -1.6659
'''

In [ ]:
# select dataset
dataset_num = 3

# get camera types
camera_names_path = os.path.join(data_path, f'dataset{dataset_num}/cameras.txt')
camera_names = get_camera_names(camera_names_path)
print(camera_names)

# load camera positions (x,y,z)
cam_pos = np.loadtxt(os.path.join(data_path, f'dataset{dataset_num}/camera-locations/campos.txt'))


# synchronization parameters between cameras:

# time scale (alpha) matrix
alpha_matrix = np.array([
    [1.0000, 0.5005, 0.4960, 0.4171, 0.5000, 0.8341],
    [1.9982, 1.0000, 0.9910, 0.8333, 0.9990, 1.6667],
    [2.0163, 1.0091, 1.0000, 0.8409, 1.0081, 1.6819],
    [2.3978, 1.2000, 1.1892, 1.0000, 1.1988, 2.0000],
    [2.0001, 1.0010, 0.9919, 0.8342, 1.0000, 1.6683],
    [1.1989, 0.6000, 0.5946, 0.5000, 0.5994, 1.0000],
])

# time shift (beta) matrix
beta_matrix = np.array([
    [0.00,     1013.95,  546.98,  251.16,  961.02,  137.51],
    [-2026.04,    0.00, -457.83, -593.82,  -51.96, -1552.47],
    [-1102.90,  461.99,    0.00, -208.81,  409.59,  -782.45],
    [ -602.21,  712.57,  248.32,    0.00,  659.93,  -364.81],
    [-1922.12,   52.01, -406.29, -551.00,    0.00, -1465.78],
    [ -164.85,  931.45,  465.22,  182.40,  878.60,     0.00],
])

In [ ]:
# get the intrinsics for the cameras from json files
camera_name = 'gopro3'
intrinsics_dict = get_camera_intrinsics(camera_name) 

print(intrinsics_dict)

#### Depth Estimation

In [ ]:
# select reference cam
ref_cam_num = 0

# isolate relevant rows for sync
alpha_ref = alpha_matrix[ref_cam_num, :]
beta_ref = beta_matrix[ref_cam_num, :]

print(alpha_ref)
print(beta_ref) 

In [ ]:
# decide reference camera
# get its data

# the loop through other cameras and get their point correspondences
# synchronized and undistorted
# then we have variables for all the point pairs

# total number of cameras
total_cams = len(camera_names)

# list of total_cams-1 lists with correspondences (each lists is Nx4 with x1,y1,x2,y2)
all_matches = []

# select reference cam
ref_cam_num = 0

# isolate relevant rows for sync
alpha_ref = alpha_matrix[ref_cam_num, :]
beta_ref = beta_matrix[ref_cam_num, :]

# get detection data for reference cam: frame_id, x_center, y_center 
ref_detections_path = os.path.join(data_path, f'dataset{dataset_num}/detections/cam{ref_cam_num}.txt')
ref_detections = np.loadtxt(ref_detections_path, skiprows=1)

# load intrinsics for reference
intrinsics_ref = get_camera_intrinsics(camera_names[ref_cam_num])
K1 = np.array(intrinsics_ref['K'])
dist1 = np.array(intrinsics_ref['dist_coeff'])


# iterate through all other cameras to get synchronized points
for cam_idx in range(1, total_cams):

    # get detection data for cur camera
    pathname = os.path.join(data_path, f'dataset{dataset_num}/detections/cam{cam_idx}.txt')
    detection_data = np.loadtxt(pathname, skiprows=1)

    # get sync params for cur camera
    alpha = alpha_ref[cam_idx]
    beta = beta_ref[cam_idx]

    # collect points
    cur_pts = []

    # iterate through all frames of reference camera
    for i in range(len(ref_detections)):

        # use sync info to get corresponding frame id
        j = alpha * i + beta
        j = int(round(j))
        if j < 0 or j >= len(detection_data):
            continue  # skip if out of bounds

        # get x, y coords from both cameras
        x1, y1 = ref_detections[i, 1], ref_detections[i, 2]
        x2, y2 = detection_data[j, 1], detection_data[j, 2]

        # skip if either coord is (0.0, 0.0)
        if (x1 == 0.0 and y1 == 0.0) or (x2 == 0.0 and y2 == 0.0):
            continue

        # append the correspondence
        cur_pts.append([x1, y1, x2, y2])

    # append to all points
    all_matches.append(cur_pts)


# for dataset3 there are 6 cameras
# the all_matches list has 5 lists that match the reference camera points to all other cams
print(len(all_matches))
print(len(all_matches[0]))
print(len(all_matches[1]))
print(len(all_matches[2]))
print(len(all_matches[3]))
print(len(all_matches[4]))


In [ ]:
# now we need to triangulate each camera to the reference cam

for i in range(len(all_matches)):

    # load intrinsics
    intrinsics = get_camera_intrinsics(camera_names[i+1])
    K2 = np.array(intrinsics_ref['K'])
    dist2 = np.array(intrinsics_ref['dist_coeff'])

    # get current matches
    match_list = all_matches[i]
    match_list = np.array(match_list)
    print(match_list.shape)
    
    # split the list into 2
    pts1, pts2 = match_list[:,:2], match_list[:,2:4]

    # convert to numpy arrays
    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)

    # undistort points with cv2 method
    pts1_undist = cv2.undistortPoints(pts1, K1, dist1)
    pts2_undist = cv2.undistortPoints(pts2, K2, dist2)

    # estimate essential matrix E using all (undistorted) point correspondences
    E, inlier_mask = cv2.findEssentialMat(pts1_undist, pts2_undist, focal=1.0, pp=(0., 0.), method=cv2.RANSAC, prob=0.999, threshold=1e-3)

    # filter points by inlier mask
    pts1_undist = pts1_undist[inlier_mask.ravel() == 1]
    pts2_undist = pts2_undist[inlier_mask.ravel() == 1]

    # recover rotation R and translation t
    _, R, t, mask_pose = cv2.recoverPose(E, pts1_undist, pts2_undist)

    # build projection matrices
    P1 = np.hstack((np.eye(3), np.zeros((3, 1))))  # first camera at origin
    P2 = np.hstack((R, t))  # second camera with R, t relative to first

    # triangulate 3D points
    points_4d_h = cv2.triangulatePoints(P1, P2, pts1_undist.reshape(2, -1), pts2_undist.reshape(2, -1))
    points_3d = (points_4d_h[:3, :] / points_4d_h[3, :]).T  # convert from homogeneous
    points_3d = np.array(points_3d)

    # filter points with positive depth (in front of cameras)
    valid = points_3d[:,2] > 0
    points_3d = points_3d[valid]



In [ ]:
# TWO CAMERAS!!!!!


# first lets try the triangulation on the ground truth detections
# and double check if the frames align

# maybe start with 2 cams
cam_A = 0
cam_B = 1

# load intrinsics
intrinsics_A = get_camera_intrinsics(camera_names[cam_A])
intrinsics_B = get_camera_intrinsics(camera_names[cam_B])
K1 = np.array(intrinsics_A['K'])
dist1 = np.array(intrinsics_A['dist_coeff'])
K2 = np.array(intrinsics_B['K'])
dist2 = np.array(intrinsics_B['dist_coeff'])

# get detection data: frame_id, x_center, y_center 
data_A_path = os.path.join(data_path, f'dataset{dataset_num}/detections/cam{cam_A}.txt')
data_B_path = os.path.join(data_path, f'dataset{dataset_num}/detections/cam{cam_B}.txt')
data_A = np.loadtxt(data_A_path, skiprows=1)
data_B = np.loadtxt(data_B_path, skiprows=1)

# get sync params
alpha = alpha_matrix[cam_A, cam_B]
beta = beta_matrix[cam_A, cam_B]

# collect corresponding pairs of points
pts1 = []
pts2 = []

# iterate through frames of reference camera
for frame_id in range(len(data_A)):

    # use sync info to get corresponding frame id from other camera
    match_id = alpha * frame_id + beta
    match_id = int(round(match_id))
    if match_id < 0 or match_id >= len(data_B):
        continue  # skip if out of bounds

    # get x, y coords from both cameras
    x1, y1 = data_A[frame_id, 1], data_A[frame_id, 2]
    x2, y2 = data_B[match_id, 1], data_B[match_id, 2]

    # skip if either point in the pair is 0.0
    if x1 == 0.0 or y1 == 0.0 or x2 == 0.0 or y2 == 0.0:
        continue

    pts1.append([x1, y1])
    pts2.append([x2, y2])

# convert to numpy arrays
pts1 = np.array(pts1, dtype=np.float32)
pts2 = np.array(pts2, dtype=np.float32)


# undistort points with cv2 method
pts1_undist = cv2.undistortPoints(pts1, K1, dist1, P=None)
pts2_undist = cv2.undistortPoints(pts2, K2, dist2, P=None)

# look into this??? P variable, dont know if points should be normalized or not
# with no projection matrix P, points are normalized
# with P=K, points are in pixel coordinates


# estimate essential matrix E using all (undistorted) point correspondences
# select threshold parameter according to pixel or normalized
E, inlier_mask = cv2.findEssentialMat(pts1_undist, pts2_undist, focal=1.0, pp=(0., 0.), method=cv2.RANSAC, prob=0.999, threshold=1e-3)
# focal=1.0, pp=(0., 0.)



# filter points by inlier mask
pts1_undist = pts1_undist[inlier_mask.ravel() == 1]
pts2_undist = pts2_undist[inlier_mask.ravel() == 1]


# recover rotation R and translation t
_, R, t, mask_pose = cv2.recoverPose(E, pts1_undist, pts2_undist)

# build projection matrices
P1 = np.hstack((np.eye(3), np.zeros((3, 1))))  # first camera at origin
P2 = np.hstack((R, t))  # second camera with R, t relative to first

# triangulate 3D points
points_4d_h = cv2.triangulatePoints(P1, P2, pts1_undist.reshape(2, -1), pts2_undist.reshape(2, -1))
points_3d = (points_4d_h[:3, :] / points_4d_h[3, :]).T  # convert from homogeneous
points_3d = np.array(points_3d)

# filter points with positive depth (in front of cameras)
valid = points_3d[:,2] > 0
points_3d = points_3d[valid]

# save to text file
np.savetxt("output.txt", points_3d)

print(f"Triangulated {points_3d.shape[0]} points.")


In [ ]:

# transform points to world coordinates
# if we had the rotation and translation of cam_A in world coords

# Example: known world camera centers
C1_world = np.array(cam_pos[cam_A])
C2_world = np.array(cam_pos[cam_B])

# From your recovered pose
t = t.reshape(3)
C1_recon = np.zeros(3)
C2_recon = t  # because in your triangulation, first cam is at origin

# Compute distances
d_real = np.linalg.norm(C2_world - C1_world)
d_recon = np.linalg.norm(C2_recon)

# Compute scale
scale = d_real / d_recon

# Apply scale to everything
t_scaled = t * scale
points_3d_scaled = points_3d * scale

print(points_3d_scaled.shape)

# save to text file
np.savetxt("output_scaled.txt", points_3d_scaled)


In [ ]:
# defo a way to vectorize this for all points

# need to be consistent between normalized and pixel coords

3589

In [ ]:
# display the 3d points
trajectory_path = "output_scaled.txt"
plot_3d_trajectory_interactive(trajectory_path)

In [ ]:
# If CSV has header
data_1 = np.genfromtxt("trajectory.csv", delimiter=",", skip_header=1)
data_2 = np.genfromtxt("trajectory_1.csv", delimiter=",", skip_header=1)

# chose second camera
cam_num_2 = 1
# get sync params
alpha_2 = alpha_ref[cam_num_2]
beta_2 = beta_ref[cam_num_2]


# iterate through frames of reference camera
for frame_id in range(len(data)):

    # use sync info to get corresponding frame id from other camera
    j = alpha_2 * frame_id + beta_2
    if j < 0 or j >= len(data_2):
        continue  # skip if out of bounds

    # get x, y coords from both cameras
    x1, y1 = data_1[frame_id, 1], data_1[frame_id, 2]
    x2, y2 = data_2[int(j), 1], data_2[int(j), 2]

    # undistort points using cv2 method
    ray1 = cv2.undistortPoints(np.array([[[x1, y1]]], dtype=np.float32), K1, dist1, P=K1)
    ray2 = cv2.undistortPoints(np.array([[[x2, y2]]], dtype=np.float32), K2, dist2, P=K2)

    # transform rays to world coordinates
    

    # triangulate 3d point from rays


In [ ]:
# try to match the points in two overlapping frames

data_path = './data/drone-tracking-datasets/'

def get_first_frame(video_path):

    cap = cv2.VideoCapture(video_path)
    ret, sample_frame = cap.read()
    cap.release()
    sample_frame_rgb = cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB)
    #plt.imshow(sample_frame_rgb)
    #plt.show()

    return sample_frame_rgb


video_path = os.path.join(data_path, f'dataset3/cam1.mp4')
cam1_frame = get_first_frame(video_path)

video_path = os.path.join(data_path, f'dataset3/cam2.mp4')
cam2_frame = get_first_frame(video_path)

In [ ]:
print(cam1_frame.shape)
print(cam2_frame.shape)

In [ ]:
# back to the fundamentals

# matched points in pixel coordinates
pts1 = np.array([...], dtype=np.float32)  # Nx2
pts2 = np.array([...], dtype=np.float32)  # Nx2

# camera intrinsics
K1 = np.array([[fx1, 0, cx1],
               [0, fy1, cy1],
               [0,   0,   1]], dtype=np.float64)
K2 = np.array([[fx2, 0, cx2],
               [0, fy2, cy2],
               [0,   0,   1]], dtype=np.float64)

# normalize points
pts1_norm = cv2.undistortPoints(pts1.reshape(-1,1,2), K1, None)
pts2_norm = cv2.undistortPoints(pts2.reshape(-1,1,2), K2, None)

# compute essential matrix directly
E, mask = cv2.findEssentialMat(pts1_norm, pts2_norm, method=cv2.RANSAC, prob=0.999, threshold=1.0)

# recover pose (rotation, translation)
_, R, t, mask_pose = cv2.recoverPose(E, pts1_norm, pts2_norm)

print("Rotation:\n", R)
print("Translation direction:\n", t)

In [ ]:
# run the detection on all videos and save to file
data_path = './data/drone-tracking-datasets/'
dataset_num = 3
num_cams = 6

# select tracker
tracker = "sort.yaml"  # or 'bytetrack.yaml', 'strongsort.yaml'

for cam_num in range(num_cams):
    video_path = os.path.join(data_path, f'dataset{dataset_num}/cam{cam_num}.mp4')
    csv_file = os.path.join('dataset3_result_detections', f'detections_cam{cam_num}.txt')

    results = model.predict(source=video_path, tracker=tracker, save=False, stream=True)

    trajectory = []  # list to store (frame_id, x_center, y_center)

    for frame_id, result in enumerate(results, start=1):
        if len(result.boxes) > 0:
            # Take first detection (assuming single drone)
            box = result.boxes[0].xywh[0]  # [x_center, y_center, width, height] in pixels
            x_center, y_center = box[0].item(), box[1].item()
        else:
            # If no detection, log 0
            x_center, y_center = 0.0, 0.0

        trajectory.append([frame_id, x_center, y_center])

    # save to CSV
    with open(csv_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["frame_id", "x_center", "y_center"])
        writer.writerows(trajectory)

    print(f"Trajectory saved to {csv_file}")